In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/reuters_health.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,586282503981375488,Thu Apr 09 21:40:16 +0000 2015,Los Angeles closes 500 medical marijuana shops...
1,586278524748750848,Thu Apr 09 21:24:27 +0000 2015,U.S. cuts poultry export forecast as deadly bi...
2,586273441801699328,Thu Apr 09 21:04:15 +0000 2015,Fears over Roundup herbicide residues prompt p...
3,586254712523096068,Thu Apr 09 19:49:50 +0000 2015,Liberia watchdog says some Ebola funds unaccou...
4,586243976333787137,Thu Apr 09 19:07:10 +0000 2015,Diabetes devices may interfere with avalanche ...


In [6]:
# -------------------------------------------------
frame.shape

(4719, 4)

In [7]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Los Angeles closes 500 medical marijuana shops, but hundreds remain '

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://reut.rs/1CvkXmm
http://reut.rs/1abCAQY
http://reut.rs/1DpfmCU
http://reut.rs/1Dp1vfF
http://reut.rs/1DoTDLl
http://reut.rs/1DoRX4s
http://reut.rs/1CuvM85
http://reut.rs/1CufHPU
http://reut.rs/1Cu6UgG
http://reut.rs/1DolhYO
http://reut.rs/1Cu2dU6
http://reut.rs/1CtdtAf
http://reut.rs/1Ct7r2B
http://reut.rs/1CrMAwn
http://reut.rs/1CrDIH4
http://reut.rs/1CqEV1z
http://reut.rs/1DlT2K8
http://reut.rs/1CqyhIC
http://reut.rs/1DlA8Dq
http://reut.rs/1CpZ6g0
http://reut.rs/1DlfOCa
http://reut.rs/1CpVnPu
http://reut.rs/1CpKux1
http://reut.rs/1Dl7UZo
http://reut.rs/1DkWkxs
http://reut.rs/1CpqB9m
http://reut.rs/1Co8ror
http://reut.rs/1Cn9e9f
http://reut.rs/1Cl0YGG
http://reut.rs/1CkO1g2
http://reut.rs/1CkEiXh
http://reut.rs/1Dh1oTj
http://reut.rs/1DgTTMg
http://reut.rs/1CkhlDC
http://reut.rs/1DgKYuc
http://reut.rs/1CjK0Zs
http://reut.rs/1CjpTKK
http://reut.rs/1DfWwhh
http://reut.rs/1DfQYDH
http://reut.rs/1CiqkVF
http://reut.rs/1DfIImZ
http://reut.rs/1Dfs4UJ
http://reut.rs/1DfgxET
http://reut

http://reut.rs/1F7wNq6
http://reut.rs/1F7lgat
http://reut.rs/1F7fVQg
http://reut.rs/1yHwFJA
http://reut.rs/1Atrp1z
http://reut.rs/1F6XBXs
http://reut.rs/1AtfhO6
http://reut.rs/1AtchBw
http://reut.rs/1F6IddM
http://reut.rs/1At9CYz
http://reut.rs/1F6IddS
http://reut.rs/1At0yCM
http://reut.rs/1F6znwI
http://reut.rs/1AsXnuT
http://reut.rs/1AsJZXN
http://reut.rs/1F6i6DP
http://reut.rs/1AsE30X
http://reut.rs/1F63GmZ
http://reut.rs/1yHwFJA
http://reut.rs/1AssqXL
http://reut.rs/1AspWZn
http://reut.rs/1ArNCgE
http://reut.rs/1Ar7511
http://reut.rs/1Ar0CmC
http://reut.rs/1F3Ez4m
http://reut.rs/1F3Ci9a
http://reut.rs/1F3v0T7
http://reut.rs/1Aq0Vhx
http://reut.rs/1F3b56B
http://reut.rs/1F378Px
http://reut.rs/1BSRqD3
http://reut.rs/16i92PY
http://reut.rs/1Apwl7M
http://reut.rs/1F2XAE9
http://reut.rs/1F2LvPc
http://reut.rs/1Apaopl
http://reut.rs/1F2IApN
http://reut.rs/1yCwmQb
http://reut.rs/1Ap32lM
http://reut.rs/1F2Egqu
http://reut.rs/1AojGBW
http://reut.rs/1F2cjz9
http://reut.rs/1F22MIf
http://reut

http://reut.rs/1yuNet8
http://reut.rs/1xHboDw
http://reut.rs/1xH2eHe
http://reut.rs/1xGWn4I
http://reut.rs/1yuqJV1
http://reut.rs/1yu86Az
http://reut.rs/1yu423k
http://reut.rs/1ytQFQL
http://reut.rs/1r0DFxx
http://reut.rs/1ytyhY7
http://reut.rs/1xGdUtK
http://reut.rs/1xGda7U
http://reut.rs/1ytlDIB
http://reut.rs/1yt2MgQ
http://reut.rs/1yrADXt
http://reut.rs/1xF3Dy2
http://reut.rs/1yrcMHo
http://reut.rs/1yqe6ui
http://reut.rs/1xE7Fqh
http://reut.rs/1xE4XBb
http://reut.rs/1yqadW5
http://reut.rs/1ypRj1z
http://reut.rs/1ypufQy
http://reut.rs/1yplwOi
http://reut.rs/1xDv1MV
http://reut.rs/1xDq32P
http://reut.rs/1yp6cBa
http://reut.rs/1yoXnHv
http://reut.rs/1xDcqjY
http://reut.rs/1xD8LTq
http://reut.rs/1xD3knk
http://reut.rs/1yomzOv
http://reut.rs/1yoa712
http://reut.rs/1xCJRTR
http://reut.rs/1yo5uUX
http://reut.rs/1xCl02C
http://reut.rs/1xChUeO
http://reut.rs/1xBWvSU
http://reut.rs/1ymxqbJ
http://reut.rs/1xBAq76
http://reut.rs/1xBrE9g
http://reut.rs/1xAZV8u
http://reut.rs/1ykWmjL
http://reut

http://reut.rs/10FIDco
http://reut.rs/1vHNALa
http://reut.rs/10FIDcq
http://reut.rs/10Fu9t1
http://reut.rs/1vGzLfT
http://reut.rs/10ET7c8
http://reut.rs/1s0ZHUB
http://reut.rs/10EkLGh
http://reut.rs/1vFJvqT
http://reut.rs/1vFcyuw
http://reut.rs/10DCxcA
http://reut.rs/1vETw7G
http://reut.rs/1vESaK2
http://reut.rs/10D02SY
http://reut.rs/1vDNC6R
http://reut.rs/10CSAax
http://reut.rs/1vDH5sQ
http://reut.rs/10BMvLt
http://reut.rs/10BKmj2
http://reut.rs/1vCcETG
http://reut.rs/1vBAZsS
http://reut.rs/10BeBGJ
http://reut.rs/1vBqqpE
http://reut.rs/1vB7CHd
http://reut.rs/1vAeIeY
http://reut.rs/10zHWBA
http://reut.rs/1vz15gf
http://reut.rs/10zjvnK
http://reut.rs/1vyhCRt
http://reut.rs/10ySMrr
http://reut.rs/1vxTsXn
http://reut.rs/10ySMru
http://reut.rs/10yE7fZ
http://reut.rs/10yvpy3
http://reut.rs/1vwFu80
http://reut.rs/10y18j4
http://reut.rs/1vsLIWm
http://reut.rs/10qheek
http://reut.rs/1vnnX21
http://reut.rs/10pZhfW
http://reut.rs/1vmC33C
http://reut.rs/1vlZB8Z
http://t.co/8J3iYq
http://reut.rs/

http://reut.rs/1tLUOBd
http://reut.rs/1sVUdJ7
http://reut.rs/1sVdSZr
http://reut.rs/1sV1yZt
http://reut.rs/1tJCrgg
http://reut.rs/1sUOS4D
http://reut.rs/1sUKSBb
http://reut.rs/1sUKSkC
http://reut.rs/1tITxe0
http://reut.rs/1tIENfe
http://reut.rs/1tICIjD
http://reut.rs/1tHpjYY
http://reut.rs/1sSEqL1
http://reut.rs/1tH3nxc
http://reut.rs/1tGVGqK
http://reut.rs/1tGVI1Z
http://reut.rs/1tGCeu9
http://reut.rs/1tFg2ki
http://reut.rs/1tEfhaZ
http://reut.rs/1txtvKG
http://reut.rs/1sJ64Kc
http://reut.rs/1sInAhK
http://reut.rs/1twCHio
http://reut.rs/1sIbKUW
http://reut.rs/1sI2kc6
http://reut.rs/1sHTJ9g
http://reut.rs/1sHMVIE
http://reut.rs/1tvUYwx
http://reut.rs/1tvUVAL
http://reut.rs/1sHu803
http://reut.rs/1tvnRZu
http://reut.rs/1sG6XTO
http://reut.rs/1tucXU3
http://reut.rs/1sFvJ6I
http://reut.rs/1tu4eB5
http://reut.rs/1tthIND
http://reut.rs/1sEoE6e
http://reut.rs/1tsV7km
http://reut.rs/1sE08lR
http://reut.rs/1tsQjv4
http://reut.rs/1tsqqvB
http://reut.rs/1tsmWcq
http://reut.rs/1sD2SQn
http://reut

...

http://reut.rs/1ovoHEd
Family of Dallas nurse Vinson says she no longer showing Ebola http://reut.rs/1ovlepc


http://reut.rs/1ovlepc
Some U.S. hospitals weigh withholding care to Ebola patients http://reut.rs/1CXLuJK


http://reut.rs/1CXLuJK
Hospital patients rarely wash their hands, may spread disease http://reut.rs/1ov8u1J


http://reut.rs/1ov8u1J
'Parents are Key' to success of National Teen Driver Safety Week, CDC says http://reut.rs/1CXuEdX


http://reut.rs/1CXuEdX
Obama says cautiously more optimistic about Ebola situation in U.S. http://reut.rs/1ouYBkF


http://reut.rs/1ouYBkF
Americans eating less trans fat, but still too much: study http://reut.rs/1ouUgOg


http://reut.rs/1ouUgOg
Extra-depth shoes may help alleviate foot pain for older people http://reut.rs/1ouQdSe


http://reut.rs/1ouQdSe
More research, regulation needed on energy drinks: study http://reut.rs/1ouMLqM


http://reut.rs/1ouMLqM
U.S. tightens Ebola monitoring for West African visitors: http://reut.rs/1sQ4lE4 htt

Outbreak of Ebola-like Marburg fever kills man in Uganda http://reut.rs/10BMvLt


http://reut.rs/10BMvLt
Sri Lanka temporarily halts some Fonterra product sales after illness http://reut.rs/10BKmj2


http://reut.rs/10BKmj2
New Jersey boy's death is 1st linked directly to enterovirus: CNN http://reut.rs/1vCcETG


http://reut.rs/1vCcETG
Nebraska hospital prepares for arrival of Ebola patient http://reut.rs/1vBAZsS


http://reut.rs/1vBAZsS
Ebola patient in Dallas 'fighting for his life': U.S. health official http://reut.rs/10BeBGJ


http://reut.rs/10BeBGJ
Ebola patient in Dallas turns critical, no new U.S. cases http://reut.rs/1vBqqpE


http://reut.rs/1vBqqpE
High risk Ebola could reach France and UK by end-October, scientists calculate http://reut.rs/1vB7CHd


http://reut.rs/1vB7CHd
Aid workers ask where was WHO in Ebola outbreak? http://reut.rs/1vAeIeY


http://reut.rs/1vAeIeY
Ebola ruled out for ill passenger escorted off plane in Newark http://reut.rs/10zHWBA


http://reut.rs/10zHWBA




http://reut.rs/1qALkWE
Fourth Ebola patient to arrive in U.S. for care http://reut.rs/1qAxkMG


http://reut.rs/1qAxkMG
Suspected Ebola case investigated in Italy http://reut.rs/1rzKAwX


http://reut.rs/1rzKAwX
U.S. government spending bill to contain new funds to fight Ebola http://reut.rs/1rx6XTP


http://reut.rs/1rx6XTP
Another Ebola patient coming to Atlanta hospital http://reut.rs/1rwLzxZ


http://reut.rs/1rwLzxZ
Starbucks goes coconuts in latest nondairy milk test http://reut.rs/1rwDBF7


http://reut.rs/1rwDBF7
Bullying from brothers and sisters may be harmful http://reut.rs/1qylaUg


http://reut.rs/1qylaUg
Diabetics pay little attention to salt warnings: study http://reut.rs/1rwqnbp


http://reut.rs/1rwqnbp
Top U.S. health adviser wants end to partisan fighting over Obamacare http://reut.rs/1rwmjYJ


http://reut.rs/1rwmjYJ
Homeland Security botched pandemic preparedness : U.S. report http://reut.rs/1qy4lJ9


http://reut.rs/1qy4lJ9
Ebola spreads exponentially in Liberia, many mo

Consignment of experimental Ebola drug arrives in Liberia http://reut.rs/1sU4lBA


http://reut.rs/1sU4lBA
Fish-eaters may keep more gray matter in old age http://reut.rs/1oNzxTT


http://reut.rs/1oNzxTT
NewLink Genetics Corp says an Ebola vaccine trial could start in weeks. A Reuters exclusive: http://reut.rs/VkeWL7


http://reut.rs/VkeWL7
Vertex to end sales of hepatitis C drug Incivek http://reut.rs/1oNmDFp


http://reut.rs/1oNmDFp
Injecting bacteria shrinks tumors in dogs and one patient: study http://reut.rs/1oNeFwd


http://reut.rs/1oNeFwd
More middle-aged U.S. women having babies outside marriage: CDC http://reut.rs/1oNeFvZ


http://reut.rs/1oNeFvZ
Exclusive: NewLink says Ebola vaccine trial could start in weeks http://reut.rs/1sTv7tZ


http://reut.rs/1sTv7tZ
U.S. government advances development of BioCryst's potential Ebola drug http://reut.rs/1sTgUxh


http://reut.rs/1sTgUxh
WHO reports 128 new Ebola cases, 45 deaths in West Africa http://reut.rs/1sTfSBn


http://reut.rs/1sTfSB



http://reut.rs/1r3rXTF
Polio still threatens Middle East after Syria, Iraq cases: U.N. http://reut.rs/1r2NziT


http://reut.rs/1r2NziT
Wawona Packing recalls fruit over listeria concerns http://reut.rs/1na4CAs


http://reut.rs/1na4CAs
U.S. appeals court throws out IRS rule on Obamacare subsidies http://reut.rs/1na4CAj


http://reut.rs/1na4CAj
Germany allows seriously ill patients to grow their own cannabis http://reut.rs/1n9WdwY


http://reut.rs/1n9WdwY
Parts of Chinese city in quarantine after plague death: Xinhua http://reut.rs/1n9Ikil


http://reut.rs/1n9Ikil
Celgene drug can drive HIV out of hiding: study http://reut.rs/1r1TWDd


http://reut.rs/1r1TWDd
India's Lupin, U.S. firms weigh bids for GSK's mature drugs: sources http://reut.rs/1n9yTzc


http://reut.rs/1n9yTzc
India faces crisis over dwindling numbers of girls, U.N. says http://reut.rs/1n9fZIS


http://reut.rs/1n9fZIS
Britain plans new measures to prevent female genital mutilation http://reut.rs/1n95jKn


http://reut.rs/1n

In [8]:
# -------------------------------------------------
frame.head()

,tweetid,date,title,url
0,586282503981375488,Thu Apr 09 21:40:16 +0000 2015,Los Angeles closes 500 medical marijuana shops...,http://reut.rs/1CvkXmm
1,586278524748750848,Thu Apr 09 21:24:27 +0000 2015,U.S. cuts poultry export forecast as deadly bi...,http://reut.rs/1abCAQY
2,586273441801699328,Thu Apr 09 21:04:15 +0000 2015,Fears over Roundup herbicide residues prompt p...,http://reut.rs/1DpfmCU
3,586254712523096068,Thu Apr 09 19:49:50 +0000 2015,Liberia watchdog says some Ebola funds unaccou...,http://reut.rs/1Dp1vfF
4,586243976333787137,Thu Apr 09 19:07:10 +0000 2015,Diabetes devices may interfere with avalanche ...,http://reut.rs/1DoTDLl


In [9]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Los Angeles closes 500 medical marijuana shops, but hundreds remain '

In [ ]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class="StandardArticleBody_body"]/p/text()',
               '',
               '//div[@class="BylineBar_byline"]/span/a/text()'
               ]

In [ ]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
list_url = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
fetch_contents(list_url, df_temp,list_scrapt)

In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape